In [1]:
import numpy as np
import pandas as pd
import pulp

<img src="./images/example_lp_formulation.jpg" width=500 height=500 align="left"/>

## Define Problem

In [2]:
prob = pulp.LpProblem("ProductMix",pulp.LpMaximize)

In [3]:
prob

ProductMix:
MAXIMIZE
None
VARIABLES

## Create decision variables

In [4]:
products = ['Desks', 'Chairs', 'Bookcases']

In [5]:
product_dvs = {}
for product in products:
    var = pulp.LpVariable(name=product,lowBound=0,)
    product_dvs[product] = var

In [6]:
product_dvs

{'Desks': Desks, 'Chairs': Chairs, 'Bookcases': Bookcases}

## Cutting hours capacity constraint

In [7]:
cutting_hours = {
    'Desks' : 0.8,
    'Chairs' : 0.4,
    'Bookcases' : 0.4
}
available_cutting_hours = 100

In [8]:
expression = pulp.LpAffineExpression()
for product in products:
    expression += cutting_hours[product] * product_dvs[product]

In [9]:
expression = expression <= available_cutting_hours
cutting_hours_constraint = pulp.LpConstraint(expression,name="CuttingHoursCapacity")

In [10]:
cutting_hours_constraint

0.4*Bookcases + 0.4*Chairs + 0.8*Desks + -100.0 = 0

In [11]:
prob += cutting_hours_constraint

## Man hours capacity constraint

In [12]:
man_hours = {
    'Desks' : 5,
    'Chairs' : 3,
    'Bookcases' : 3
}
available_man_hours = 650

In [13]:
expression = pulp.LpAffineExpression()
for product in products:
    expression += man_hours[product] * product_dvs[product]

In [14]:
expression = expression <= available_man_hours
man_hours_constraint = pulp.LpConstraint(expression,name="ManHoursCapacity")

In [15]:
man_hours_constraint

3*Bookcases + 3*Chairs + 5*Desks + -650 = 0

In [16]:
prob += man_hours_constraint

## Warehouse capacity constraint

In [17]:
floor_space_requirements = {
    'Desks' : 1,
    'Chairs' : 0.7,
    'Bookcases' : 1.1
}
available_floor_space = 140

In [18]:
expression = pulp.LpAffineExpression()
for product in products:
    expression += floor_space_requirements[product] * product_dvs[product]

In [19]:
expression = expression <= available_floor_space
floor_space_constraint = pulp.LpConstraint(expression,name="WarehouseCapacity")

In [20]:
floor_space_constraint

1.1*Bookcases + 0.7*Chairs + 1*Desks + -140.0 = 0

In [21]:
prob += floor_space_constraint

In [22]:
prob

ProductMix:
MAXIMIZE
None
SUBJECT TO
CuttingHoursCapacity: 0.4 Bookcases + 0.4 Chairs + 0.8 Desks = 100

ManHoursCapacity: 3 Bookcases + 3 Chairs + 5 Desks = 650

WarehouseCapacity: 1.1 Bookcases + 0.7 Chairs + Desks = 140

VARIABLES
Bookcases Continuous
Chairs Continuous
Desks Continuous

## Objective Function

## Maximize Profit
## $30 X_{Desks} + 16 X_{Chairs} + 25 X_{Bookcases}$

In [23]:
product_profit = {
    'Desks' : 30,
    'Chairs' : 16,
    'Bookcases' : 5
}

In [24]:
obj_expression = pulp.LpAffineExpression()
for product in products:
    obj_expression += product_profit[product] * product_dvs[product]

In [25]:
obj_expression

5*Bookcases + 16*Chairs + 30*Desks + 0

In [26]:
prob += obj_expression

In [27]:
prob

ProductMix:
MAXIMIZE
5*Bookcases + 16*Chairs + 30*Desks + 0
SUBJECT TO
CuttingHoursCapacity: 0.4 Bookcases + 0.4 Chairs + 0.8 Desks = 100

ManHoursCapacity: 3 Bookcases + 3 Chairs + 5 Desks = 650

WarehouseCapacity: 1.1 Bookcases + 0.7 Chairs + Desks = 140

VARIABLES
Bookcases Continuous
Chairs Continuous
Desks Continuous

In [28]:
solver = pulp.get_solver('GLPK_CMD')

In [29]:
prob.solve(solver=solver)

1

In [30]:
for product in products:
    value = product_dvs[product].value()
    print(f"{product} = {value}")

Desks = 100.0
Chairs = 37.5
Bookcases = 12.5


In [31]:
for c_name in prob.constraints:
    print(c_name,prob.constraints[c_name].value())

CuttingHoursCapacity 0.0
ManHoursCapacity 0.0
WarehouseCapacity 1.7763568394002505e-15


# Solvers

In [32]:
solver_list = pulp.listSolvers()
solver_list

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'CPLEX_DLL',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'PULP_CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD']

In [33]:
solver_list = pulp.listSolvers(onlyAvailable=True)
solver_list

['GLPK_CMD', 'SCIP_CMD']

In [34]:
solver = pulp.get_solver('SCIP_CMD')

In [35]:
solver

In [36]:
solver.to_dict()

{'solver': 'SCIP_CMD',
 'mip': True,
 'msg': True,
 'keepFiles': False,
 'maxNodes': None}

# How does it work ?

<img src="./images/model_solver_eco_system.jpg" width=600 height=500 align="left"/>

In [37]:
prob.writeLP("product_mix.lp")

[Bookcases, Chairs, Desks]

In [38]:
prob.writeMPS("product_mix.mps")

[Bookcases, Chairs, Desks]